# Analizar recibos con Form Recognizer

![Un robot sujetando un recibo](./images/receipt_analysis.jpg)

En el campo de inteligencia artificial de Computer Vision, el reconocimiento óptico de caracteres (OCR) se suele usar para leer documentos impresos o escritos a mano. A menudo, el texto se extrae de los documentos en un formato que se puede procesar o analizar.

Un caso más avanzado del uso de un OCR sería la extracción de información de formularios, como pedidos o facturas, con una comprensión semántica del significado de los campos del formulario. El servicio **Form Recognizer** está diseñado especialmente para este TIPO de situaciones de IA.

## Ver un recibo

En este ejemplo, usará el modelo integrado de Form Recognizer para analizar recibos.

Haga clic en el botón **Run cell** (&#9655;) a la izquierda de la celda siguiente para ejecutarla y ver un ejemplo de un recibo que analizaremos con Form Recognizer.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Cargar y mostrar una imagen de confirmación
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Creación de un recurso de Form Recognizer

Cree un recurso de Form Recognizer en su suscripción de Azure:

1. En la pestaña de otro explorador, abra Azure Portal (https://portal.azure.com) e inicie sesión con su cuenta de Microsoft.
2. Seleccione **+ Crear un recurso** y busque *Form Recognizer*.
3. En la lista de servicios, seleccione **Form Recognizer**.
4. En la hoja **Form Recognizer**, seleccione **Crear**.
5. En la hoja **Crear**, escriba la siguiente información y haga clic en **Crear**.
   - **Nombre**: un nombre exclusivo para su servicio
   - **Suscripción**: su suscripción de Azure
   - **Región**: cualquier región disponible
   - **Plan de tarifa**: F0
   - **Grupo de recursos**: el grupo de recursos utilizado anteriormente
   - **Confirmo que he leído y comprendido la notificación siguiente**: seleccionado.
6. Espere a que se cree el servicio.
7. Consulte su nuevo servicio Form Recognizer en Azure Portal. En la página **Keys and Endpoint**, copie los valores de **Key1** y **Endpoint** y péguelos en el siguiente código, en sustitución de **YOUR_FORM_KEY** y **YOUR_FORM_ENDPOINT**.

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## Análisis de un recibo

Ya está listo para usar Form Recognizer para analizar un recibo.

In [ ]:
import os
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Crear un cliente del servicio Form Recognizer
form_recognizer_client = FormRecognizerClient(endpoint=form_endpoint, credential=AzureKeyCredential(form_key))

try:
    print("Analyzing receipt...")
    # Get the receipt image file
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')

    # Envíe los datos de archivo a Form Recognizer
    with open(image_path, "rb") as f:
        analyze_receipt = form_recognizer_client.begin_recognize_receipts(receipt=f)
    
    # Obtenga los resultados
    receipt_data = analyze_receipt.result()

    # Imprima los datos extraídos para el primer (y único) recibo
    receipt = receipt_data[0]
    receipt_type = receipt.fields.get("ReceiptType")
    if receipt_type:
        print("Receipt Type: {}".format(receipt_type.value))
    merchant_address = receipt.fields.get("MerchantAddress")
    if merchant_address:
        print("Merchant Address: {}".format(merchant_address.value))
    merchant_phone = receipt.fields.get("MerchantPhoneNumber")
    if merchant_phone:
        print("Merchant Phone: {}".format(merchant_phone.value))
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print("Transaction Date: {}".format(transaction_date.value))
    print("Receipt items:")
    items = receipt.fields.get("Items")
    if items:
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("\tItem #{}".format(idx+1))
            item_name = item.value.get("Name")
            if item_name:
                print("\t - Name: {}".format(item_name.value))
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print("\t - Price: {}".format(item_total_price.value))
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print("Subtotal: {} ".format(subtotal.value))
    tax = receipt.fields.get("Tax")
    if tax:
        print("Tax: {}".format(tax.value))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {}".format(total.value))

except Exception as ex:
    print('Error:', ex)


Tenga en cuenta que Form Recognizer es capaz de interpretar los datos del formulario, además de identificar correctamente la dirección y número de teléfono del comerciante, la fecha y hora de la transacción, así como los elementos de línea, los subtotales, impuestos y cuantías totales.

## Más información

Para obtener más información sobre el servicio Form Recognizer, consulte la [documentación del servicio Form Recognizer](https://docs.microsoft.com/es-es/azure/cognitive-services/form-recognizer/index)